# 데이터 불러오기

In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv('train_1.csv')
test = pd.read_csv('test_1.csv')

# 라이브러리

In [15]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

#개봉계절 전처리
def get_month(x):
    if x[5:7] == '12' or x[5:7] == '01' or x[5:7] =='02':
        return 1
    elif x[5:7] == '03' or x[5:7] =='04' or x[5:7] =='05':
        return 2
    elif x[5:7] == '06' or x[5:7] =='07' or x[5:7] =='08':
        return 3
    elif x[5:7] == '09' or x[5:7] =='10' or x[5:7] =='11':
        return 4

#시간 전처리
def get_time(x):
    if 115 <= x:
        return 5
    elif 100 <= x:
        return 6
    else:
        return 7

#관람 등급 전처리
def get_rat(x):
    if "15" in x:
        return 8
    elif "12" in x:
        return 9
    elif '전체' in x:
        return 10
    else:
        return 11

#스태프 전처리
def get_staff(x):
    if 250 <=x:
        return 12
    else:
        return 13



# 데이터 전처리

## 사용하지 않을 데이터 제거

In [16]:
# 감독명 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

# 제목 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

# 감독 제작 영화 수 제거
train = train.drop(['dir_prev_num'],axis= 1)
test = test.drop(['dir_prev_num'],axis= 1)

# 배우 제거
train = train.drop(['num_actor'],axis= 1)
test = test.drop(['num_actor'],axis= 1)

#### 배급사 제거 #####
#train = train.drop(['distributor'],axis= 1)
#test = test.drop(['distributor'],axis= 1)

## 결측치 채워넣기 (중간단계)

In [17]:
test.fillna(0, inplace = True)
train.fillna(0, inplace = True)

## 장르 전처리

In [18]:
#장르 중위값 계산
train.groupby('genre').box_off_num.mean().sort_values()

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [19]:
#순위가 맞는지 확인
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

## 개봉일 전처리

In [20]:
#계절구분
pre_month = train.release_time.apply(get_month)
train['pre_month'] = pre_month
month_dum = pd.get_dummies(train['pre_month'])
train = pd.concat([train, month_dum], axis='columns')

pre_month = test.release_time.apply(get_month)
test['pre_month'] = pre_month
month_dum = pd.get_dummies(test['pre_month'])
test = pd.concat([test, month_dum], axis='columns')

## 상영시간 전처리

In [21]:
# 115분 110분 기준
pre_time = train.time.apply(get_time)
train['pre_time'] = pre_time
time_dum = pd.get_dummies(train['pre_time'])
train = pd.concat([train, time_dum], axis='columns')

pre_time = train.time.apply(get_time)
test['pre_time'] = pre_time
time_dum = pd.get_dummies(test['pre_time'])
test = pd.concat([test, time_dum], axis='columns')

## 관람등급 전처리

In [22]:
pre_rat = train.screening_rat.apply(get_rat)
train['pre_rat'] = pre_rat
rat_dum = pd.get_dummies(train['pre_rat'])
train = pd.concat([train, rat_dum], axis='columns')

pre_rat = train.screening_rat.apply(get_rat)
test['pre_rat'] = pre_rat
rat_dum = pd.get_dummies(test['pre_rat'])
test = pd.concat([test, rat_dum], axis='columns')

## 스태프 전처리

In [23]:
pre_staff = train.num_staff.apply(get_staff)
train['pre_staff'] = pre_staff
staff_dum = pd.get_dummies(train['pre_staff'])
train = pd.concat([train, staff_dum], axis='columns')

pre_staff = train.num_staff.apply(get_staff)
test['pre_staff'] = pre_staff
staff_dum = pd.get_dummies(test['pre_staff'])
test = pd.concat([test, staff_dum], axis='columns')

## 더미 전 데이터 드랍
#### (개봉일, 상영시간, 관람등급, 스태프)

In [24]:
train = train.drop(['genre', 'release_time', 'time', 'screening_rat', 'num_staff', 'pre_month', 'pre_time', 'pre_rat', 'pre_staff'],axis = 1)
test =  test.drop(['genre', 'release_time', 'time', 'screening_rat', 'num_staff', 'pre_month', 'pre_time', 'pre_rat', 'pre_staff'],axis = 1)

## 임시 코드
train = train.drop(['distributor'], axis=1)
test = test.drop(['distributor'], axis=1)

## 결측치 채워넣기

In [25]:
test.fillna(0, inplace = True)
train.fillna(0, inplace = True)

## 확인하기

In [26]:
train

,dir_prev_bfnum,box_off_num,genre_rank,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00,23398,11,0,0,0,1,0,0,1,0,0,0,1,0,1
1,1161602.50,7072501,12,0,0,0,1,1,0,0,0,0,0,1,1,0
2,220775.25,6959083,11,0,0,1,0,1,0,0,1,0,0,0,1,0
3,23894.00,217866,9,0,0,1,0,0,1,0,0,0,1,0,0,1
4,1.00,483387,9,0,0,0,1,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,3833.00,1475091,8,0,0,1,0,0,1,0,0,0,0,1,1,0
596,496061.00,1716438,8,0,1,0,0,1,0,0,1,0,0,0,1,0
597,0.00,2475,7,0,0,0,1,0,0,1,0,0,0,1,0,1
598,0.00,2192525,12,0,1,0,0,0,1,0,1,0,0,0,1,0


In [27]:
test

,dir_prev_bfnum,genre_rank,1,2,3,4,5,6,7,8,9,10,11,12,13
0,3.005290e+05,12,1,0,0,0,0,0,1,0,0,0,1,0,1
1,3.427002e+05,5,1,0,0,0,1,0,0,0,0,0,1,1,0
2,4.206611e+06,8,1,0,0,0,1,0,0,1,0,0,0,1,0
3,6.913420e+05,11,1,0,0,0,0,1,0,0,0,1,0,0,1
4,3.173800e+04,7,1,0,0,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2.590000e+03,8,0,0,0,1,0,0,1,1,0,0,0,0,1
239,0.000000e+00,2,0,0,0,1,0,1,0,0,0,0,1,0,1
240,0.000000e+00,12,0,0,0,1,0,0,1,1,0,0,0,0,1
241,5.069900e+04,8,0,0,0,1,0,0,1,0,0,1,0,0,1


## 모델정의 학습

In [28]:
train_x = train.drop(['box_off_num'],axis= 1)
train_y = train['box_off_num']

In [29]:
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)

RandomForestRegressor()

In [30]:
pred = model.predict(test)

## 제출 파일 생성

In [31]:
submission = pd.read_csv('submission.csv')
submission['box_off_num'] = pred
submission

,title,box_off_num
0,용서는 없다,3.002560e+04
1,아빠가 여자를 좋아해,2.594336e+06
2,하모니,4.211666e+06
3,의형제,5.308163e+05
4,평행 이론,6.392113e+05
...,...,...
238,해에게서 소년에게,2.659597e+03
239,울보 권투부,1.874226e+04
240,어떤살인,2.808485e+04
241,말하지 못한 비밀,1.673747e+04


In [32]:
submission.to_csv('Finally_movies.csv',index = False)

In [1]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'Finally_movies.csv', 
'8daef5351dafa78817e0fdf3301ccd6efc2863efb8a6f875903bae099c106176', 
'235536', 
'rgm', 
'RandomForestRegressor' )

exception occurs. 예외가 발생했습니다 [Errno 2] No such file or directory: 'team2_movies.csv'
